In [68]:
import requests
import h5py
import os
import cv2
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import numpy as np
import pandas as pd
from glob import glob

In [69]:
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
images_path = "./images/256_ObjectCategories"
h5_file_output = "./datasets/256_images.h5"


def save_images_to_h5():
    start = dt.datetime.now()
    dtype = h5py.special_dtype(vlen=np.dtype('uint8'))

    for directory in next(os.walk("./images/256_ObjectCategories"))[1]:
        SOURCE_IMAGES = os.path.join(images_path, directory)
        images = glob(os.path.join(SOURCE_IMAGES, "*.jpg"))
        
        NUM_IMAGES = len(images)
        SHAPE = (HEIGHT, WIDTH, CHANNELS)
        
        with h5py.File(h5_file_output, "w") as wf:
 
            with h5py.File(h5_file_output, "a") as hf:
                for i, img in enumerate(images):
                    image = cv2.imread(img)
                    image = cv2.resize(image, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)

                    X_set = hf.create_dataset(
                        name="X" + str(i),
                        data=image,
                        shape=(HEIGHT, WIDTH, CHANNELS),
                        maxshape=(HEIGHT, WIDTH, CHANNELS),
                        compression="gzip",
                        compression_opts=9
                    )

                    y_set = hf.create_dataset(
                        name="y" + str(i) + "_" + directory,
                        data=directory,
                        shape=(1,),
                        maxshape=(1,),
                        compression="gzip",
                        compression_opts=9
                    )

In [70]:
save_images_to_h5()

In [75]:
hf = h5py.File(h5_file_output, "r")
print(np.array(hf["X0"][:]))

[[[104 117 138]
  [110 116 135]
  [132 130 142]
  ...
  [111 131 156]
  [110 120 153]
  [ 96 115 136]]

 [[118 120 153]
  [119 125 146]
  [133 132 148]
  ...
  [114 134 165]
  [113 128 165]
  [ 90 103 135]]

 [[ 97 111 133]
  [119 126 136]
  [138 130 153]
  ...
  [102 125 151]
  [105 123 155]
  [ 99 113 142]]

 ...

 [[156 159 163]
  [149 154 156]
  [146 152 151]
  ...
  [148 147 126]
  [161 160 135]
  [156 152 126]]

 [[165 164 169]
  [152 154 155]
  [147 152 151]
  ...
  [161 155 137]
  [154 144 126]
  [145 141 114]]

 [[162 162 162]
  [153 157 151]
  [151 155 160]
  ...
  [147 148 122]
  [152 153 124]
  [150 146 122]]]
